<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
    Machine learning <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2024<br>
<font color=3C99D size=5>
    Practical Assignment 5 - NLP - Transformer & Bert <br>
</div>
<div dir=ltr align=center>
<font color=0CBCDF size=4>
   &#x1F349; Masoud Tahmasbi  &#x1F349;  &#x1F353; Arash Ziyaei &#x1F353;
<br>
<font color=0CBCDF size=4>
   &#x1F335; Amirhossein Akbari  &#x1F335;
</div>

____

<font color=9999FF size=4>
&#x1F388; Full Name : Amir Mohammad Mahfoozi
<br>
<font color=9999FF size=4>
&#x1F388; Student Number : 401106469

<font color=0080FF size=3>
This notebook covers two key topics. First, we implement a transformer model from scratch and apply it to a specific task. Second, we fine-tune the BERT model using LoRA for efficient adaptation to a downstream task.
</font>
<br>

**Note:**
<br>
<font color=66B2FF size=2>In this notebook, you are free to use any function or model from PyTorch to assist with the implementation. However, TensorFlow is not permitted for this exercise. This ensures consistency and alignment with the tools being focused on.</font>
<br>
<font color=red size=3>**Run All Cells Before Submission**</font>: <font color=FF99CC size=2>Before saving and submitting your notebook, please ensure you run all cells from start to finish. This practice guarantees that your notebook is self-consistent and can be evaluated correctly by others.</font>

# Section 1: Transformer

The transformer architecture consists of two main components: an encoder and a decoder. Each of these components is made up of multiple layers that include self-attention mechanisms and feedforward neural networks. The self-attention mechanism is central to the transformer, as it enables the model to assess the importance of different words in a sentence by considering their relationships with one another.


In this assignment, you should design a transformer model from scratch. You are required to implement the Encoder and Decoder components of a Transformer model.

In [ ]:
# Importing libraries

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

# Math
import math

# HuggingFace libraries
!pip install datasets

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

# Pathlib
from pathlib import Path

# typing
from typing import Any

# Library for progress bars in loops
from tqdm import tqdm

# Importing library of warnings
import warnings

## Part 1: Input Embeddings
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">When we observe the Transformer architecture image above, we can see that the Embeddings represent the first step of both blocks.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>InputEmbedding</code> class below is responsible for converting the input text into numerical vectors of <code>d_model</code> dimensions. To prevent that our input embeddings become extremely small, we normalize them by multiplying them by the $\sqrt{d_{model}}$.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the image below, we can see how the embeddings are created. First, we have a sentence that gets split into tokens—we will explore what tokens are later on—. Then, the token IDs—identification numbers—are transformed into the embeddings, which are high-dimensional vectors.</p>

In [ ]:
import torch
import torch.nn as nn
import math

class TokenEmbeddings(nn.Module):
    def __init__(self, embedding_dim: int, vocab_size: int) -> None:
        super().__init__()
        self.embedding_dim = embedding_dim
        self.token_vocab_size = vocab_size
        self.embedding_layer = self._initialize_embedding()

    def _initialize_embedding(self) -> nn.Embedding:
        return nn.Embedding(self.token_vocab_size, self.embedding_dim)

    def scale_embeddings(self, embeddings: torch.Tensor) -> torch.Tensor:
        return embeddings * math.sqrt(self.embedding_dim)

    def forward(self, input_indices: torch.Tensor) -> torch.Tensor:
        embedded_tokens = self.embedding_layer(input_indices)
        return self.scale_embeddings(embedded_tokens)


## Part 2: positional encoding
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the original paper, the authors add the positional encodings to the input embeddings at the bottom of both the encoder and decoder blocks so the model can have some information about the relative or absolute position of the tokens in the sequence. The positional encodings have the same dimension $d_{model}$ as the embeddings, so that the two vectors can be summed and we can combine the semantic content from the word embeddings and positional information from the positional encodings.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>PositionalEncoding</code> class below, we will create a matrix of positional encodings <code>pe</code> with dimensions <code>(seq_len, d_model)</code>. We will start by filling it with $0$s.We will then apply the sine function to even indices of the positional encoding matrix while the cosine function is applied to the odd ones.</p>

<p style="
    margin-bottom: 5;
    font-size: 22px;
    font-weight: 300;
    font-family: 'Helvetica Neue', sans-serif;
    color: #000000;
  ">
    \begin{equation}
    \text{Odd Indices } (2i + 1): \quad \text{PE(pos, } 2i + 1) = \cos\left(\frac{\text{pos}}{10000^{2i / d_{model}}}\right)
    \end{equation}
</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We apply the sine and cosine functions because it allows the model to determine the position of a word based on the position of other words in the sequence, since for any fixed offset $k$, $PE_{pos + k}$ can be represented as a linear function of $PE_{pos}$. This happens due to the properties of sine and cosine functions, where a shift in the input results in a predictable change in the output.</p>

In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):

    def __init__(self, embedding_dim: int, sequence_length: int, dropout_rate: float) -> None:
        super().__init__()
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.dropout_layer = nn.Dropout(dropout_rate)
        self.positional_matrix = self._generate_positional_encoding()
        self.register_buffer('positional_encoding', self.positional_matrix)

    def _generate_positional_encoding(self) -> torch.Tensor:
        position_matrix = torch.zeros(self.sequence_length, self.embedding_dim)
        position_indices = torch.arange(0, self.sequence_length, dtype=torch.float).unsqueeze(1)
        frequency_term = torch.exp(torch.arange(0, self.embedding_dim, 2).float() * (-math.log(10000.0) / self.embedding_dim))
        position_matrix[:, 0::2] = torch.sin(position_indices * frequency_term)
        position_matrix[:, 1::2] = torch.cos(position_indices * frequency_term)
        return position_matrix.unsqueeze(0)

    def add_positional_encoding(self, input_tensor: torch.Tensor) -> torch.Tensor:
        return input_tensor + self.positional_encoding[:, :input_tensor.shape[1], :].requires_grad_(False)

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        input_with_encoding = self.add_positional_encoding(input_tensor)
        return self.dropout_layer(input_with_encoding)


## Part 3: layer normalization
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">When we look at the encoder and decoder blocks, we see several normalization layers called <b><i>Add &amp; Norm</i></b>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>LayerNormalization</code> class below performs layer normalization on the input data. During its forward pass, we compute the mean and standard deviation of the input data. We then normalize the input data by subtracting the mean and dividing by the standard deviation plus a small number called epsilon to avoid any divisions by zero. This process results in a normalized output with a mean 0 and a standard deviation 1.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will then scale the normalized output by a learnable parameter <code>alpha</code> and add a learnable parameter called <code>bias</code>. The training process is responsible for adjusting these parameters. The final result is a layer-normalized tensor, which ensures that the scale of the inputs to layers in the network is consistent.</p>

In [ ]:
import torch
import torch.nn as nn

class LayerNormalization(nn.Module):

    def __init__(self, num_features: int, epsilon: float = 1e-6) -> None:
        super().__init__()
        self.epsilon = epsilon
        self.scale_factor = nn.Parameter(torch.ones(num_features))
        self.offset = nn.Parameter(torch.zeros(num_features))

    def compute_statistics(self, input_tensor: torch.Tensor) -> tuple:
        mean = input_tensor.mean(dim=-1, keepdim=True)
        std = input_tensor.std(dim=-1, keepdim=True)
        return mean, std

    def normalize(self, input_tensor: torch.Tensor, mean: torch.Tensor, std: torch.Tensor) -> torch.Tensor:
        return (input_tensor - mean) / (std + self.epsilon)

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        mean, std = self.compute_statistics(input_tensor)
        normalized_tensor = self.normalize(input_tensor, mean, std)
        return self.scale_factor * normalized_tensor + self.offset

## Part 4: Feed Forward Network
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the fully connected feed-forward network, we apply two linear transformations with a ReLU activation in between. We can mathematically represent this operation as:</p>

<p style="
    margin-bottom: 5;
    font-size: 22px;
    font-weight: 300;
    font-family: 'Helvetica Neue', sans-serif;
    color: #000000;
  ">
    \begin{equation}
    \text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
    \end{equation}
</p>


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">$W_1$ and $W_2$ are the weights, while $b_1$ and $b_2$ are the biases of the two linear transformations.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>FeedForwardBlock</code> below, we will define the two linear transformations—<code>self.linear_1</code> and <code>self.linear_2</code>—and the inner-layer <code>d_ff</code>. The input data will first pass through the <code>self.linear_1</code> transformation, which increases its dimensionality from <code>d_model</code> to <code>d_ff</code>. The output of this operation passes through the ReLU activation function, which introduces non-linearity so the network can learn more complex patterns, and the <code>self.dropout</code> layer is applied to mitigate overfitting. The final operation is the <code>self.linear_2</code> transformation to the dropout-modified tensor, which transforms it back to the original <code>d_model</code> dimension.</p>

In [ ]:
import torch
import torch.nn as nn

class FeedForwardBlock(nn.Module):

    def __init__(self, input_dim: int, hidden_dim: int, dropout_rate: float) -> None:
        super().__init__()
        self.input_to_hidden = nn.Linear(input_dim, hidden_dim)
        self.hidden_dropout = nn.Dropout(dropout_rate)
        self.hidden_to_output = nn.Linear(hidden_dim, input_dim)

    def apply_activation_and_dropout(self, hidden_tensor: torch.Tensor) -> torch.Tensor:
        activated_tensor = torch.relu(hidden_tensor)
        return self.hidden_dropout(activated_tensor)

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        hidden_tensor = self.input_to_hidden(input_tensor)
        activated_tensor = self.apply_activation_and_dropout(hidden_tensor)
        return self.hidden_to_output(activated_tensor)

## Part 5: Multi Head Attention
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The Multi-Head Attention is the most crucial component of the Transformer. It is responsible for helping the model to understand complex relationships and patterns in the data.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The image below displays how the Multi-Head Attention works. It doesn't include <code>batch</code> dimension because it only illustrates the process for one single sentence.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The Multi-Head Attention block receives the input data split into queries, keys, and values organized into matrices $Q$, $K$, and $V$. Each matrix contains different facets of the input, and they have the same dimensions as the input.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We then linearly transform each matrix by their respective weight matrices $W^Q$, $W^K$, and $W^V$. These transformations will result in new matrices $Q'$, $K'$, and $V'$, which will be split into smaller matrices corresponding to different heads $h$, allowing the model to attend to information from different representation subspaces in parallel. This split creates multiple sets of queries, keys, and values for each head.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Finally, we concatenate every head into an $H$ matrix, which is then transformed by another weight matrix $W^o$ to produce the multi-head attention output, a matrix $MH-A$ that retains the input dimensionality.</p>

In [ ]:
import torch
import torch.nn as nn
import math

class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, model_dim: int, num_heads: int, dropout_rate: float) -> None:
        super().__init__()
        self.model_dim = model_dim
        self.num_heads = num_heads
        assert model_dim % num_heads == 0, "model_dim must be divisible by num_heads"

        self.head_dim = model_dim // num_heads
        self.query_weight = nn.Linear(model_dim, model_dim, bias=False)
        self.key_weight = nn.Linear(model_dim, model_dim, bias=False)
        self.value_weight = nn.Linear(model_dim, model_dim, bias=False)
        self.output_weight = nn.Linear(model_dim, model_dim, bias=False)
        self.dropout_layer = nn.Dropout(dropout_rate)

    @staticmethod
    def compute_attention_scores(query_tensor, key_tensor, value_tensor, mask_tensor, dropout_layer):
        scaling_factor = query_tensor.shape[-1] ** 0.5
        scores = query_tensor @ key_tensor.transpose(-2, -1) / scaling_factor
        if mask_tensor is not None:
            scores.masked_fill_(mask_tensor == 0, -1e9)
        scores = scores.softmax(dim=-1)
        if dropout_layer is not None:
            scores = dropout_layer(scores)
        return scores @ value_tensor, scores

    def reshape_for_heads(self, tensor):
        batch_size, seq_length, _ = tensor.shape
        return tensor.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, tensor):
        batch_size, num_heads, seq_length, head_dim = tensor.shape
        return tensor.transpose(1, 2).contiguous().view(batch_size, seq_length, num_heads * head_dim)

    def forward(self, query_input, key_input, value_input, mask_tensor):
        query_tensor = self.reshape_for_heads(self.query_weight(query_input))
        key_tensor = self.reshape_for_heads(self.key_weight(key_input))
        value_tensor = self.reshape_for_heads(self.value_weight(value_input))

        attention_output, self.attention_scores = self.compute_attention_scores(
            query_tensor, key_tensor, value_tensor, mask_tensor, self.dropout_layer
        )

        combined_output = self.combine_heads(attention_output)
        return self.output_weight(combined_output)

## Part 6: Residual Connection
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">When we look at the architecture of the Transformer, we see that each sub-layer, including the <i>self-attention</i> and <i>Feed Forward</i> blocks, adds its output to its input before passing it to the <i>Add &amp; Norm</i> layer. This approach integrates the output with the original input in the <i>Add &amp; Norm</i> layer. This process is known as the skip connection, which allows the Transformer to train deep networks more effectively by providing a shortcut for the gradient to flow through during backpropagation.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>ResidualConnection</code> class below is responsible for this process.</p>

In [ ]:
import torch.nn as nn

class ResidualConnection(nn.Module):
    def __init__(self, input_size: int, dropout_rate: float) -> None:
        super().__init__()
        self.dropout_layer = nn.Dropout(dropout_rate)
        self.layer_norm = LayerNormalization(input_size)

    def forward(self, input_tensor, sublayer_fn):
        normed_tensor = self._normalize_input(input_tensor)
        sublayer_output = self._apply_sublayer(normed_tensor, sublayer_fn)
        return self._apply_residual_connection(input_tensor, sublayer_output)

    def _normalize_input(self, input_tensor):
        return self.layer_norm(input_tensor)

    def _apply_sublayer(self, normed_tensor, sublayer_fn):
        return sublayer_fn(normed_tensor)

    def _apply_residual_connection(self, original_input, sublayer_output):
        return original_input + self.dropout_layer(sublayer_output)


## Part 7: Encoder
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will now build the encoder. We create the <code>EncoderBlock</code> class, consisting of the Multi-Head Attention and Feed Forward layers, plus the residual connections.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the original paper, the Encoder Block repeats six times. We create the <code>Encoder</code> class as an assembly of multiple <code>EncoderBlock</code>s. We also add layer normalization as a final step after processing the input through all its blocks.</p>

In [ ]:
import torch.nn as nn

class EncoderBlock(nn.Module):
    def __init__(self, input_size: int, self_attention: MultiHeadAttentionBlock, feed_forward: FeedForwardBlock, dropout_rate: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention
        self.feed_forward_block = feed_forward
        self.residual_connections = nn.ModuleList([ResidualConnection(input_size, dropout_rate) for _ in range(2)])

    def forward(self, input_tensor, source_mask):
        attention_output = self._apply_self_attention(input_tensor, source_mask)
        feed_forward_output = self._apply_feed_forward(attention_output)
        return feed_forward_output

    def _apply_self_attention(self, input_tensor, source_mask):
        return self.residual_connections[0](input_tensor, lambda x: self.self_attention_block(x, x, x, source_mask))

    def _apply_feed_forward(self, input_tensor):
        return self.residual_connections[1](input_tensor, self.feed_forward_block)


In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size: int, encoder_layers: nn.ModuleList) -> None:
        super().__init__()
        self.encoder_layers = encoder_layers
        self.output_norm = LayerNormalization(input_size)

    def forward(self, input_tensor, mask):
        output_tensor = self._apply_layers(input_tensor, mask)
        return self._normalize_output(output_tensor)

    def _apply_layers(self, input_tensor, mask):
        for layer in self.encoder_layers:
            input_tensor = layer(input_tensor, mask)
        return input_tensor

    def _normalize_output(self, output_tensor):
        return self.output_norm(output_tensor)


## Part 8: Decoder
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Similarly, the Decoder also consists of several DecoderBlocks that repeat six times in the original paper. The main difference is that it has an additional sub-layer that performs multi-head attention with a <i>cross-attention</i> component that uses the output of the Encoder as its keys and values while using the Decoder's input as queries.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">For the Output Embedding, we can use the same <code>InputEmbeddings</code> class we use for the Encoder. You can also notice that the self-attention sub-layer is <i>masked</i>, which restricts the model from accessing future elements in the sequence.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will start by building the <code>DecoderBlock</code> class, and then we will build the <code>Decoder</code> class, which will assemble multiple <code>DecoderBlock</code>s.</p>

In [ ]:
import torch.nn as nn

class DecoderBlock(nn.Module):
    def __init__(self, input_size: int, self_attention: MultiHeadAttentionBlock, cross_attention: MultiHeadAttentionBlock, feed_forward: FeedForwardBlock, dropout_rate: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention
        self.cross_attention_block = cross_attention
        self.feed_forward_block = feed_forward
        self.residual_connections = nn.ModuleList([ResidualConnection(input_size, dropout_rate) for _ in range(3)])

    def forward(self, input_tensor, encoder_output, source_mask, target_mask):
        attention_output = self._apply_self_attention(input_tensor, target_mask)
        cross_attention_output = self._apply_cross_attention(attention_output, encoder_output, source_mask)
        feed_forward_output = self._apply_feed_forward(cross_attention_output)
        return feed_forward_output

    def _apply_self_attention(self, input_tensor, target_mask):
        return self.residual_connections[0](input_tensor, lambda x: self.self_attention_block(x, x, x, target_mask))

    def _apply_cross_attention(self, input_tensor, encoder_output, source_mask):
        return self.residual_connections[1](input_tensor, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, source_mask))

    def _apply_feed_forward(self, input_tensor):
        return self.residual_connections[2](input_tensor, self.feed_forward_block)


In [ ]:
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, input_size: int, decoder_layers: nn.ModuleList) -> None:
        super().__init__()
        self.decoder_layers = decoder_layers
        self.output_norm = LayerNormalization(input_size)

    def forward(self, input_tensor, encoder_output, source_mask, target_mask):
        decoded_tensor = self._apply_decoder_layers(input_tensor, encoder_output, source_mask, target_mask)
        return self._normalize_output(decoded_tensor)

    def _apply_decoder_layers(self, input_tensor, encoder_output, source_mask, target_mask):
        for layer in self.decoder_layers:
            input_tensor = layer(input_tensor, encoder_output, source_mask, target_mask)
        return input_tensor

    def _normalize_output(self, decoded_tensor):
        return self.output_norm(decoded_tensor)


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">You can see in the Decoder image that after running a stack of <code>DecoderBlock</code>s, we have a Linear Layer and a Softmax function to the output of probabilities. The <code>ProjectionLayer</code> class below is responsible for converting the output of the model into a probability distribution over the <i>vocabulary</i>, where we select each output token from a vocabulary of possible tokens.</p>

In [ ]:
import torch.nn as nn

class ProjectionLayer(nn.Module):
    def __init__(self, model_dim: int, vocab_size: int) -> None:
        super().__init__()
        self.linear_projection = nn.Linear(model_dim, vocab_size)

    def forward(self, input_tensor) -> None:
        projected_tensor = self._project_input(input_tensor)
        log_probabilities = self._apply_log_softmax(projected_tensor)
        return log_probabilities

    def _project_input(self, input_tensor):
        return self.linear_projection(input_tensor)

    def _apply_log_softmax(self, projected_tensor):
        return nn.functional.log_softmax(projected_tensor, dim=-1)


## Part 9: Building the Transformer

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We finally have every component of the Transformer architecture ready. We may now construct the Transformer by putting it all together.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>Transformer</code> class below, we will bring together all the components of the model's architecture.</p>

In [ ]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src_input, src_mask):
        embedded_src = self._apply_source_embedding(src_input)
        positionally_encoded_src = self._apply_source_positional_encoding(embedded_src)
        return self.encoder(positionally_encoded_src, src_mask)

    def decode(self, encoder_output, src_mask, tgt_input, tgt_mask):
        embedded_tgt = self._apply_target_embedding(tgt_input)
        positionally_encoded_tgt = self._apply_target_positional_encoding(embedded_tgt)
        return self.decoder(positionally_encoded_tgt, encoder_output, src_mask, tgt_mask)

    def project(self, decoder_output):
        return self.projection_layer(decoder_output)

    def _apply_source_embedding(self, src_input):
        return self.src_embed(src_input)

    def _apply_target_embedding(self, tgt_input):
        return self.tgt_embed(tgt_input)

    def _apply_source_positional_encoding(self, embedded_src):
        return self.src_pos(embedded_src)

    def _apply_target_positional_encoding(self, embedded_tgt):
        return self.tgt_pos(embedded_tgt)


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The architecture is finally ready. We now define a function called <code>build_transformer</code>, in which we define the parameters and everything we need to have a fully operational Transformer model for the task of <b>machine translation</b>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will set the same parameters as in the original paper, <a href = "https://arxiv.org/pdf/1706.03762.pdf"><i>Attention Is All You Need</i></a>, where $d_{model}$ = 512, $N$ = 6, $h$ = 8, dropout rate $P_{drop}$ = 0.1, and $d_{ff}$ = 2048.</p>

In [ ]:
import torch.nn as nn

def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    src_embed = self._create_embedding_layer(d_model, src_vocab_size)
    tgt_embed = self._create_embedding_layer(d_model, tgt_vocab_size)

    src_pos = self._create_positional_encoding_layer(d_model, src_seq_len, dropout)
    tgt_pos = self._create_positional_encoding_layer(d_model, tgt_seq_len, dropout)

    encoder_blocks = self._create_encoder_blocks(d_model, h, N, dropout, d_ff)
    decoder_blocks = self._create_decoder_blocks(d_model, h, N, dropout, d_ff)

    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    self._initialize_parameters(transformer)

    return transformer

def _create_embedding_layer(self, d_model: int, vocab_size: int) -> InputEmbeddings:
    return InputEmbeddings(d_model, vocab_size)

def _create_positional_encoding_layer(self, d_model: int, seq_len: int, dropout: float) -> PositionalEncoding:
    return PositionalEncoding(d_model, seq_len, dropout)

def _create_encoder_blocks(self, d_model: int, h: int, N: int, dropout: float, d_ff: int) -> list:
    encoder_blocks = []
    for _ in range(N):
        self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)
    return encoder_blocks

def _create_decoder_blocks(self, d_model: int, h: int, N: int, dropout: float, d_ff: int) -> list:
    decoder_blocks = []
    for _ in range(N):
        self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, self_attention_block, cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
    return decoder_blocks

def _initialize_parameters(self, transformer: Transformer) -> None:
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)


The model is now ready to be trained!

## Part 10: Tokenizer

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Tokenization is a crucial preprocessing step for our Transformer model. In this step, we convert raw text into a number format that the model can process.  </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">There are several Tokenization strategies. We will use the <i>word-level tokenization</i> to transform each word in a sentence into a token.</p>

<center>
    <img src = "https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F8d5e749c-b0bd-4496-85a1-9b4397ad935f_1400x787.jpeg" width = 800, height= 800>
<p style = "font-size: 16px;
            font-family: 'Georgia', serif;
            text-align: center;
            margin-top: 10px;">Different tokenization strategies. Source: <a href = "https://shaankhosla.substack.com/p/talking-tokenization">shaankhosla.substack.com</a>.</p>
</center>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">After tokenizing a sentence, we map each token to an unique integer ID based on the created vocabulary present in the training corpus during the training of the tokenizer. Each integer number represents a specific word in the vocabulary.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Besides the words in the training corpus, Transformers use special tokens for specific purposes. These are some that we will define right away:</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>• [UNK]:</b> This token is used to identify an unknown word in the sequence.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>• [PAD]:</b> Padding token to ensure that all sequences in a batch have the same length, so we pad shorter sentences with this token. We use attention masks to <i>"tell"</i> the model to ignore the padded tokens during training since they don't have any real meaning to the task.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>•  [SOS]:</b> This is a token used to signal the <i>Start of Sentence</i>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>•  [EOS]:</b> This is a token used to signal the <i>End of Sentence</i>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>build_tokenizer</code> function below, we ensure a tokenizer is ready to train the model. It checks if there is an existing tokenizer, and if that is not the case, it trains a new tokenizer.</p>

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path

def build_tokenizer(config, ds, lang):
    tokenizer_path = _get_tokenizer_path(config, lang)

    if not _tokenizer_exists(tokenizer_path):
        tokenizer = _create_and_train_tokenizer(ds, lang, tokenizer_path)
    else:
        tokenizer = _load_tokenizer(tokenizer_path)

    return tokenizer

def _get_tokenizer_path(config, lang):
    return Path(config['tokenizer_file'].format(lang))

def _tokenizer_exists(tokenizer_path):
    return tokenizer_path.exists()

def _create_and_train_tokenizer(ds, lang, tokenizer_path):
    tokenizer = _initialize_tokenizer()
    trainer = _initialize_trainer()
    tokenizer.train_from_iterator(_get_all_sentences(ds, lang), trainer=trainer)
    tokenizer.save(str(tokenizer_path))
    return tokenizer

def _initialize_tokenizer():
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    return tokenizer

def _initialize_trainer():
    return WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)

def _load_tokenizer(tokenizer_path):
    return Tokenizer.from_file(str(tokenizer_path))


## Part 11: Load Dataset

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">For this task, we will use the <a href = "opus_books · Datasets at Hugging Face">OpusBooks dataset</a>, available on 🤗Hugging Face. This dataset consists of two features, <code>id</code> and <code>translation</code>. The <code>translation</code> feature contains pairs of sentences in different languages, such as Spanish and Portuguese, English and French, and so forth.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">I first tried translating sentences from English to Portuguese—my native tongue — but there are only 1.4k examples for this pair, so the results were not satisfying in the current configurations for this model. I then tried to use the English-French pair due to its higher number of examples—127k—but it would take too long to train with the current configurations. I then opted to train the model on the English-Italian pair, the same one used in the <a href = "https://youtu.be/ISNdQcPhsts?si=253J39cose6IdsLv">Coding a Transformer from scratch on PyTorch, with full explanation, training and inference
</a> video, as that was a good balance between performance and time of training.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We start by defining the <code>get_all_sentences</code> function to iterate over the dataset and extract the sentences according to the language pair defined—we will do that later.</p>

In [ ]:
def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>get_ds</code> function is defined to load and prepare the dataset for training and validation. In this function, we build or load the tokenizer, split the dataset, and create DataLoaders, so the model can successfully iterate over the dataset in batches. The result of these functions is tokenizers for the source and target languages plus the DataLoader objects.</p>

In [ ]:
import random
from torch.utils.data import DataLoader, random_split
from datasets import load_dataset

def get_ds(config):
    ds_raw = _load_raw_dataset(config)

    tokenizer_src = _build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = _build_tokenizer(config, ds_raw, config['lang_tgt'])

    train_ds, val_ds = _split_dataset(ds_raw, config)

    _print_max_sentence_lengths(ds_raw, tokenizer_src, tokenizer_tgt)

    train_dataloader = _create_dataloader(train_ds, config, shuffle=True)
    val_dataloader = _create_dataloader(val_ds, config, shuffle=False)

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt


def _load_raw_dataset(config):
    return load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')


def _build_tokenizer(config, ds_raw, lang):
    return get_or_build_tokenizer(config, ds_raw, lang)


def _split_dataset(ds_raw, config):
    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    return train_ds, val_ds


def _print_max_sentence_lengths(ds_raw, tokenizer_src, tokenizer_tgt):
    max_len_src = 0
    max_len_tgt = 0

    for item in ds_raw:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f'Max length of source sentence: {max_len_src}')
    print(f'Max length of target sentence: {max_len_tgt}')


def _create_dataloader(dataset, config, shuffle):
    return DataLoader(dataset, batch_size=config['batch_size'], shuffle=shuffle)


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We define the <code>casual_mask</code> function to create a mask for the attention mechanism of the decoder. This mask prevents the model from having information about future elements in the sequence. </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We start by making a square grid filled with ones. We determine the grid size with the <code>size</code> parameter. Then, we change all the numbers above the main diagonal line to zeros. Every number on one side becomes a zero, while the rest remain ones. The function then flips all these values, turning ones into zeros and zeros into ones. This process is crucial for models that predict future tokens in a sequence.</p>

In [ ]:
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>BilingualDataset</code> class processes the texts of the target and source languages in the dataset by tokenizing them and adding all the necessary special tokens. This class also certifies that the sentences are within a maximum sequence length for both languages and pads all necessary sentences.</p>

In [ ]:
import torch
from torch.utils.data import Dataset

class BilingualDataset(Dataset):

    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()
        self.seq_len = seq_len
        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        # Tokenize the source and target sentences
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        # Calculate padding required
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2  # excluding <s> and </s>
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1  # excluding <s>

        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        # Construct encoder, decoder input, and label tensors
        encoder_input = torch.cat([
            self.sos_token,
            torch.tensor(enc_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64)
        ], dim=0)

        decoder_input = torch.cat([
            self.sos_token,
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
        ], dim=0)

        label = torch.cat([
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
        ], dim=0)

        # Ensure that all tensors have the expected seq_len size
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,  # (seq_len)
            "decoder_input": decoder_input,  # (seq_len)
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),  # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & self._causal_mask(decoder_input.size(0)),  # (1, seq_len)
            "label": label,  # (seq_len)
            "src_text": src_text,
            "tgt_text": tgt_text,
        }

    def _causal_mask(self, seq_len):
        # Create a causal mask for the decoder (ensuring that tokens can only attend to earlier ones)
        mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0).unsqueeze(0).int()
        return mask


## Part 12: Validation Loop

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will now create two functions for the validation loop. The validation loop is crucial to evaluate model performance in translating sentences from data it has not seen during training.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will define two functions. The first function, <code>greedy_decode</code>, gives us the model's output by obtaining the most probable next token. The second function, <code>run_validation</code>, is responsible for running the validation process in which we decode the model's output and compare it with the reference text for the target sentence.</p>

In [ ]:
import torch

def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    encoder_output = model.encode(source, source_mask)
    decoder_input = torch.full((1, 1), sos_idx, dtype=torch.long, device=device)

    generated_tokens = torch.full((1, max_len), tokenizer_tgt.token_to_id('[PAD]'), dtype=torch.long, device=device)

    for i in range(max_len):
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, next_word.unsqueeze(0).unsqueeze(0)], dim=1)
        generated_tokens[:, i] = next_word
        if next_word == eos_idx:
            break

    return generated_tokens.squeeze(0)


In [ ]:
import os
import torch

def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device)
            encoder_mask = batch["encoder_mask"].to(device)

            assert encoder_input.size(0) == 1

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break

    if writer:
        metric = torchmetrics.CharErrorRate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation cer', cer, global_step)
        writer.flush()

        metric = torchmetrics.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation wer', wer, global_step)
        writer.flush()

        metric = torchmetrics.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation BLEU', bleu, global_step)
        writer.flush()


## Part 13: Training Loop

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We are ready to train our Transformer model on the OpusBook dataset for the English to Italian translation task.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We first start by defining the <code>get_model</code> function to load the model by calling the <code>build_transformer</code> function we have previously defined. This function uses the <code>config</code> dictionary to set a few parameters.</p>

In [ ]:
def get_model(config, vocab_src_len, vocab_tgt_len):
    model = build_transformer(vocab_src_len, vocab_tgt_len, config["seq_len"], config['seq_len'], d_model=config['d_model'])
    return model


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">I have mentioned the <code>config</code> dictionary several times throughout this notebook. Now, it is time to create it.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the following cell, we will define two functions to configure our model and the training process.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>get_config</code> function, we define crucial parameters for the training process. <code>batch_size</code> for the number of training examples used in one iteration, <code>num_epochs</code> as the number of times the entire dataset is passed forward and backward through the Transformer, <code>lr</code> as the learning rate for the optimizer, etc. We will also finally define the pairs from the OpusBook dataset, <code>'lang_src': 'en'</code> for selecting English as the source language and <code>'lang_tgt': 'it'</code> for selecting Italian as the target language.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>get_weights_file_path</code> function constructs the file path for saving or loading model weights for any specific epoch.</p>

In [ ]:
from pathlib import Path

def get_config():
    return {
        "batch_size": 8,
        "num_epochs": 5,
        "lr": 10**-4,
        "seq_len": 350,
        "d_model": 512,
        "datasource": 'opus_books',
        "lang_src": "en",
        "lang_tgt": "it",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

def get_weights_file_path(config, epoch: str):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

def latest_weights_file_path(config):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}*"
    weights_files = list(Path(model_folder).glob(model_filename))
    if len(weights_files) == 0:
        return None
    weights_files.sort()
    return str(weights_files[-1])


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We finally define our last function, <code>train_model</code>, which takes the <code>config</code> arguments as input. </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In this function, we will set everything up for the training. We will load the model and its necessary components onto the GPU for faster training, set the <code>Adam</code> optimizer, and configure the <code>CrossEntropyLoss</code> function to compute the differences between the translations output by the model and the reference translations from the dataset. </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Every loop necessary for iterating over the training batches, performing backpropagation, and computing the gradients is in this function. We will also use it to run the validation function and save the current state of the model.</p>

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import os

def train_model(config):
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps or torch.backends.mps.is_available() else "cpu"
    print("Using device:", device)
    if (device == 'cuda'):
        print(f"Device name: {torch.cuda.get_device_name(device.index)}")
        print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
    elif (device == 'mps'):
        print(f"Device name: <mps>")
    else:
        print("NOTE: If you have a GPU, consider using it for training.")
        print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
        print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
    device = torch.device(device)

    Path(f"{config['datasource']}_{config['model_folder']}").mkdir(parents=True, exist_ok=True)

    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
    writer = SummaryWriter(config['experiment_name'])

    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

    initial_epoch = 0
    global_step = 0
    preload = config['preload']
    model_filename = latest_weights_file_path(config) if preload == 'latest' else get_weights_file_path(config, preload) if preload else None
    if model_filename:
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
    else:
        print('No model to preload, starting from scratch')

    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    for epoch in range(initial_epoch, config['num_epochs']):
        torch.cuda.empty_cache()
        model.train()
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        for batch in batch_iterator:

            encoder_input = batch['encoder_input'].to(device)
            decoder_input = batch['decoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            decoder_mask = batch['decoder_mask'].to(device)

            encoder_output = model.encode(encoder_input, encoder_mask)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            proj_output = model.project(decoder_output)

            label = batch['label'].to(device)

            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            loss.backward()

            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

            global_step += 1

        run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)

        model_filename = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)


We can now train the model!

In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader, random_split
# from torch.optim.lr_scheduler import LambdaLR

# import warnings
# from tqdm import tqdm
# import os
# from pathlib import Path

# # Huggingface datasets and tokenizers
# from datasets import load_dataset
# from tokenizers import Tokenizer
# from tokenizers.models import WordLevel
# from tokenizers.trainers import WordLevelTrainer
# from tokenizers.pre_tokenizers import Whitespace
!pip install torchmetrics

import torchmetrics
# from torch.utils.tensorboard import SummaryWriter
# !pip install numpy==1.26.0

if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    config = get_config()
    train_model(config)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 14.9 MB/s eta 0:00:00
Using device: cuda
Device name: Tesla T4
Device memory: 14.74810791015625 GB


README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence: 309
Max length of target sentence: 274
No model to preload, starting from scratch


...

--------------------------------------------------------------------------------
    SOURCE: Alice was more and more puzzled, but she thought there was no use in saying anything more till the Pigeon had finished.
    TARGET: Alice sempre più confusa, pensò che sarebbe stato inutile dir nulla, sin che il Colombo non avesse finito.
 PREDICTED: Alice era un poco , ma non era più più che non era più di .
--------------------------------------------------------------------------------
    SOURCE: Nothing special was said during the quadrille: they talked in snatches about the Korsunskys, husband and wife, whom Vronsky very amusingly described as dear forty-year-old children, and about a proposed Stage Society, and only once did the conversation touch her to the quick – when he asked her about Levin, whether he was still in Moscow, and added that he had liked him very much.
    TARGET: Durante la quadriglia non fu detto nulla di particolare. La conversazione, smozzicata, si aggirò ora sui Ko

Processing Epoch 03: 100%|██████████| 3638/3638 [25:09<00:00,  2.41it/s, loss=5.086]


--------------------------------------------------------------------------------
    SOURCE: What could my darling do, I asked, left destitute and penniless?
    TARGET: "Mi domandavo che cosa avrebbe fatto la mia cara, povera e abbandonata.
 PREDICTED: Che cosa mi domandò , e io mi ?
--------------------------------------------------------------------------------
    SOURCE: And that you should hire horses in the village is, in the first place, unpleasant to me, and besides that, they will undertake the job but won't get you there.
    TARGET: E poi, prenderli in affitto al paese, in primo luogo mi rincresce, e poi anche se prenderanno l’incarico, non ti porteranno fin là.
 PREDICTED: E voi , che la barca è la prima volta , è stata , e che io non si , e che io non .
--------------------------------------------------------------------------------


Processing Epoch 04: 100%|██████████| 3638/3638 [25:08<00:00,  2.41it/s, loss=4.754]


--------------------------------------------------------------------------------
    SOURCE: 'It is worse than cruelty, it is – baseness, if you want to know!' Anna exclaimed in a burst of anger, and rose to go.
    TARGET: — È peggiore della crudeltà, questa è vigliaccheria, se volete saperlo — gridò Anna in uno scoppio di rabbia e, alzatasi, fece per andar via.
 PREDICTED: — È più difficile , per voi , è vero , per voi , se tu non — gridò Anna , guardandolo con un sorriso .
--------------------------------------------------------------------------------
    SOURCE: Now I am sure of it.'
    TARGET: È questo che io so, adesso.
 PREDICTED: Ora sono molto contenta .
--------------------------------------------------------------------------------


# Section 2: BERT and LoRA

Welcome to Section 2 of our Machine Learning assignment! I hope you've been enjoying the journey so far! 😊

 In this section, you will gain hands-on experience with [BERT](https://arxiv.org/abs/1810.04805) (Bidirectional Encoder Representations from Transformers) and [LoRA](https://arxiv.org/abs/2106.09685) (Low-Rank Adaptation) for text classification tasks. The section is divided into three main parts, each focusing on different aspects of NLP techniques.

## Assignment Structure

### Part 1: Data Preparation and Preprocessing
In this part, you will work with a text classification dataset. You will learn how to:
- Download and load the dataset
- Perform necessary preprocessing steps
- Implement data cleaning and transformation techniques
- Prepare the data in a format suitable for BERT training

### Part 2: Building a Small BERT Model
You will create and train a small BERT model from scratch using the Hugging Face [Transformers](https://huggingface.co/docs/transformers/en/index) library. This part will help you understand:
- The architecture of BERT
- How to configure and initialize a BERT model
- Training process and optimization
- Model evaluation and performance analysis

### Part 3: Fine-tuning with LoRA
In the final part, you will work with a pre-trained [TinyBERT](https://arxiv.org/abs/1909.10351) model and use LoRA for efficient fine-tuning. You will:
- Load a pre-trained TinyBERT model
- Implement LoRA adaptation and fine-tune the model on our classification task
- Compare the results with the previous approach

---

> **NOTE**:  
> Throughout this notebook, make an effort to include sufficient visualizations to enhance understanding:  
> - In the data processing section, display the results of your operations (e.g., show data samples or distributions after preprocessing).  
> - In the classification section, report various evaluation metrics such as accuracy, precision, recall, and F1-score to thoroughly assess your model's performance.  
> - Additionally, take a moment to compare the sizes of the models discussed in this notebook with today’s enormous models. This will help you appreciate the challenges and computational demands associated with training such massive models. 😵‍💫

---


## Part 1: Data Preparation and Preprocessing
We'll be working with the [Consumer Complaint](https://catalog.data.gov/dataset/consumer-complaint-database) dataset, which contains ***complaints*** submitted by consumers about financial products and services. Our goal is to build a classifier that can automatically identify the type of complaint based on the consumer's text description. For this task, we will work with a smaller subset of the dataset, available for download through this [link](https://drive.google.com/file/d/1SpIHksR-WzruEgUjp1SQKGG8bZPnJJoN/view?usp=sharing).

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

### 1.2 Loading the Data

In [ ]:
######################  TODO  ########################
######################  TODO  ########################
# Load the dataset
# Load the dataset
data_path = "./complaints_small.csv"
df = pd.read_csv(data_path)

# Display basic information about the dataset
print("Dataset Overview:")
print(df.head())
print("\nDataset Columns:")
print(df.columns)
print("\nDataset Shape:")
print(df.shape)

# Check for missing values
print("\nMissing Values per Column:")
print(df.isnull().sum())

######################  TODO  ########################
######################  TODO  ########################

Dataset Overview:
                                             Product  \
0  Credit reporting, credit repair services, or o...   
1                                       Student loan   
2  Credit reporting or other personal consumer re...   
3  Credit reporting, credit repair services, or o...   
4  Credit reporting or other personal consumer re...   

                        Consumer complaint narrative  
0  My credit reports are inaccurate. These inaccu...  
1  Beginning in XX/XX/XXXX I had taken out studen...  
2  I am disputing a charge-off on my account that...  
3  I did not consent to, authorize, nor benefit f...  
4  I am a federally protected consumer and I am a...  

Dataset Columns:
Index(['Product', 'Consumer complaint narrative'], dtype='object')

Dataset Shape:
(36405, 2)

Missing Values per Column:
Product                         0
Consumer complaint narrative    1
dtype: int64


### 1.3 Data Sampling and Class Distribution Analysis

Working with large datasets can be computationally intensive during development. Additionally, imbalanced class distribution can affect model performance. In this section, you'll sample the data and analyze class distributions to make informed decisions about your training dataset.

---

We'll work with a manageable portion of the data to develop and test our approach. While using the complete dataset would likely yield better results, a smaller sample allows us to prototype our solution more efficiently.


In [ ]:
######################  TODO  ########################
######################  TODO  ########################

# - Sample a portion of the complete dataset
# - Display the first few rows of your sampled dataset
# - Print the shape of your original and sampled datasets

######################  TODO  ########################
######################  TODO  ########################

sampled_df = df.sample(frac=0.2, random_state=42)  # Adjust the fraction as needed

print("Sampled Dataset Overview:")
print(sampled_df.head())

print(f"\nOriginal Dataset Shape: {df.shape}")
print(f"Sampled Dataset Shape: {sampled_df.shape}")

if 'label_column' in df.columns:  # Replace 'label_column' with the actual column name for labels
    print("\nClass Distribution in Original Dataset:")
    print(df['label_column'].value_counts(normalize=True))

    print("\nClass Distribution in Sampled Dataset:")
    print(sampled_df['label_column'].value_counts(normalize=True))
else:
    print("No label column specified. Please replace 'label_column' with the actual column name.")


Sampled Dataset Overview:
                                                 Product  \
24443  Credit reporting or other personal consumer re...   
16321  Credit reporting, credit repair services, or o...   
12657  Credit reporting, credit repair services, or o...   
31695                                           Mortgage   
8499   Credit reporting or other personal consumer re...   

                            Consumer complaint narrative  
24443  an inaccurate and unverified charge-off on my ...  
16321  There are accounts on my credit report that ar...  
12657  It has been drawn out into the open that you a...  
31695  This is the XXXX complaint I have filed agains...  
8499   I've learned that the creditor didn't provide ...  

Original Dataset Shape: (36405, 2)
Sampled Dataset Shape: (7281, 2)
No label column specified. Please replace 'label_column' with the actual column name.


---

Let's examine the distribution of ***complaints*** types in our dataset. You'll notice that some products have significantly more instances than others, and some categories are quite similar. For example:

- Multiple categories might refer to similar financial products
- Some categories might have very few examples
- Certain categories might be subcategories of others

You have two main approaches to handle this situation:

1. **Merging Similar Classes:** Identify categories that represent similar products/services and Combine them to create more robust, general categories

2. **Selecting Major Classes:** Only select the categories with sufficient representation



> You may choose any approach, but after this step, your data must include **at least five** distinct classes.



In [ ]:
######################  TODO  ########################
######################  TODO  ########################

# - Display the number of complaints in each product category
# - Identify which classes are under-represented

# - Handle class imbalance by choosing and implementing one of these approaches:
#   1. Merge similar product categories (e.g., combining related categories)
#   2. Keep only the major classes with sufficient examples

######################  TODO  ########################
######################  TODO  ########################

print("Available Columns in Dataset:")
print(df.columns)

target_column = None
for col in df.columns:
    if "product" in col.lower() or "category" in col.lower():
        target_column = col
        break

if target_column:
    print(f"\nDetected target column: {target_column}")

    print("\nClass Distribution in Original Dataset:")
    print(df[target_column].value_counts())

    class_counts = df[target_column].value_counts()
    threshold = 50  # Adjust this based on your dataset
    underrepresented_classes = class_counts[class_counts < threshold].index.tolist()

    print(f"\nUnderrepresented Classes (less than {threshold} instances):")
    print(underrepresented_classes)

    merge_mapping = {
        'Credit reporting': 'Credit Services',
        'Credit card': 'Credit Services',
        'Payday loan': 'Loans',
        'Vehicle loan': 'Loans',
    }
    df[target_column] = df[target_column].replace(merge_mapping)

    major_classes = class_counts[class_counts >= threshold].index.tolist()
    df_filtered = df[df[target_column].isin(major_classes)]

    print("\nClass Distribution After Handling Imbalance:")
    print(df_filtered[target_column].value_counts())

    unique_classes = df_filtered[target_column].nunique()
    print(f"\nNumber of Unique Classes After Processing: {unique_classes}")

else:
    print("No suitable column found for complaints classification. Please check your dataset.")


Available Columns in Dataset:
Index(['Product', 'Consumer complaint narrative'], dtype='object')

Detected target column: Product

Class Distribution in Original Dataset:
Product
Credit reporting, credit repair services, or other personal consumer reports    12470
Credit reporting or other personal consumer reports                              9795
Debt collection                                                                  4568
Mortgage                                                                         1906
Checking or savings account                                                      1728
Credit card or prepaid card                                                      1607
Credit card                                                                       970
Student loan                                                                      735
Money transfer, virtual currency, or money service                                696
Vehicle loan or lease                          

---
### 1.4 Data Encoding and Text Preprocessing

Before training our model, we need to prepare both our target labels and text data. This involves converting categorical labels into numerical format and cleaning our text data to improve model performance.

In [ ]:
######################  TODO  ########################
######################  TODO  ########################

# Label Encoding
# - Apply label encoding to convert product categories into numeric values

# Text Preprocessing
# Choose and implement preprocessing steps that you think will improve the quality of your text data.
# Here are some suggestions:

# - Remove special characters and punctuation
# - Remove very short complaints (e.g., less than 10 words)
# - Remove HTML tags if present

######################  TODO  ########################
######################  TODO  ########################
from sklearn.preprocessing import LabelEncoder
import re

text_column = "Consumer complaint narrative"
target_column = "Product"

label_encoder = LabelEncoder()
df_filtered['label_encoded'] = label_encoder.fit_transform(df_filtered[target_column])

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:")
print(label_mapping)

def preprocess_text(text):
    if pd.isnull(text):
        return ""  # Handle missing values gracefully
    text = re.sub(r'<.*?>', '', text)

    text = re.sub(r'[^a-zA-Z\s]', '', text)

    text = text.lower()

    text = re.sub(r'\s+', ' ', text).strip()

    return text

df_filtered['cleaned_text'] = df_filtered[text_column].apply(preprocess_text)

df_filtered = df_filtered[df_filtered['cleaned_text'].apply(lambda x: len(x.split()) >= 10)]

print("\nProcessed Data Sample:")
print(df_filtered[['cleaned_text', target_column, 'label_encoded']].head())

print(f"\nFinal Dataset Shape After Preprocessing: {df_filtered.shape}")



Label Mapping:
{'Bank account or service': 0, 'Checking or savings account': 1, 'Consumer Loan': 2, 'Credit card or prepaid card': 3, 'Credit reporting or other personal consumer reports': 4, 'Credit reporting, credit repair services, or other personal consumer reports': 5, 'Debt collection': 6, 'Money transfer, virtual currency, or money service': 7, 'Mortgage': 8, 'Payday loan, title loan, or personal loan': 9, 'Payday loan, title loan, personal loan, or advance loan': 10, 'Prepaid card': 11, 'Student loan': 12, 'Vehicle loan or lease': 13}

Processed Data Sample:
                                        cleaned_text  \
0  my credit reports are inaccurate these inaccur...   
1  beginning in xxxxxxxx i had taken out student ...   
2  i am disputing a chargeoff on my account that ...   
3  i did not consent to authorize nor benefit fro...   
4  i am a federally protected consumer and i am a...   

                                             Product  label_encoded  
0  Credit reporting,

## 1.5 Dataset Creation and Tokenization

For training our BERT model, we need to:
1. Create a custom Dataset class that will handle tokenization
2. Split the data into training and testing sets
3. Use BERT's tokenizer to convert text into a format suitable for the model

In [ ]:
######################  TODO  ########################
######################  TODO  ########################

from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

class ComplaintDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # Remove batch dimension
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

######################  TODO  ########################
######################  TODO  ########################

In [ ]:
######################  TODO  ########################
######################  TODO  ########################

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_filtered['cleaned_text'].tolist(),
    df_filtered['label_encoded'].tolist(),
    test_size=0.2,
    random_state=42
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = ComplaintDataset(X_train, y_train, tokenizer)
test_dataset = ComplaintDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

for batch in train_loader:
    print("\nSample Batch:")
    print("Input IDs:", batch['input_ids'].shape)
    print("Attention Mask:", batch['attention_mask'].shape)
    print("Labels:", batch['labels'].shape)
    break


######################  TODO  ########################
######################  TODO  ########################


Sample Batch:
Input IDs: torch.Size([16, 512])
Attention Mask: torch.Size([16, 512])
Labels: torch.Size([16])


## Part 2: Training a Small-Size BERT Model

In this part, we will explore how to build and train a small-sized BERT model for our classification task. Instead of using the full-sized BERT model, which is computationally expensive, we will create a smaller version using the Transformers library.

In [ ]:
######################  TODO  ########################
######################  TODO  ########################

# 1. Define your BERT model for sequence classification
#    Ensure that you set up the configuration properly (e.g., specify the number of output labels).
# 2. Print the total number of trainable parameters in the model to understand its size.

######################  TODO  ########################
######################  TODO  ########################

import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig

num_labels = len(label_mapping)  # Automatically set based on the number of classes
model_config = BertConfig.from_pretrained('bert-base-uncased', num_labels=num_labels)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=model_config)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal Trainable Parameters: {total_params}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Total Trainable Parameters: 109493006


---

Now that you have defined your model, it's time to train it!☠️

Training a model of this size can take some time, depending on the available resources. To manage this, you can train your model for just **2–3 epochs** to demonstrate progress. Here are some hints:
- **Training Metrics:** Ensure you print enough metrics, such as loss and accuracy, to track the training progress.
- **Interactive Monitoring:** Use the `tqdm` library to display the progress of your training loop in real-time.

In [ ]:
######################  TODO  ########################
######################  TODO  ########################
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0

    print(f"\nEpoch {epoch + 1}/{num_epochs} - Training:")
    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        # Move batch to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        # Calculate loss and accuracy
        loss = outputs.loss
        logits = outputs.logits
        total_loss += loss.item()

        # Calculate accuracy
        predictions = torch.argmax(logits, dim=-1)
        correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print epoch metrics
    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total_samples
    print(f"\nEpoch {epoch + 1} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


from sklearn.metrics import classification_report

# Evaluation mode
model.eval()
correct = 0
total_samples = 0
all_predictions = []
all_labels = []

print("\nEvaluating on the Test Set:")
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Track metrics
        correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print final accuracy and detailed metrics
test_accuracy = correct / total_samples
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=label_mapping.keys()))


######################  TODO  ########################
######################  TODO  ########################

Using device: cuda

Epoch 1/3 - Training:


100%|██████████| 1729/1729 [44:38<00:00,  1.55s/it]



Epoch 1 - Loss: 0.9690, Accuracy: 0.6503

Epoch 2/3 - Training:


100%|█████████▉| 1726/1729 [44:29<00:04,  1.56s/it]

## Part 3: Fine-Tuning TinyBERT with LoRA

As you have experienced, training even a small-sized BERT model can be computationally intensive and time-consuming. To address these challenges, we explore **Parameter-Efficient Fine-Tuning (PEFT)** methods, which allow us to utilize the power of large pretrained models without requiring extensive resources.

---

### **Parameter-Efficient Fine-Tuning (PEFT)**

PEFT methods focus on fine-tuning only a small portion of the model’s parameters while keeping most of the pretrained weights frozen. This drastically reduces the computational and storage requirements while leveraging the rich knowledge embedded in pretrained models.

One popular PEFT method is LoRA (Low-Rank Adaptation).

- **What is LoRA?**

LoRA introduces a mechanism to fine-tune large language models by injecting small low-rank matrices into the model's architecture. Instead of updating all parameters during training, LoRA trains these small matrices while keeping the majority of the original parameters frozen.  This is achieved as follows:

1. **Frozen Weights**: The pretrained weights of the model, represented as a weight matrix $ W \in \mathbb{R}^{d \times k} $, remain **frozen** during fine-tuning.

2. **Low-Rank Decomposition**:
   Instead of directly updating $ W $, LoRA introduces two trainable matrices, $ A \in \mathbb{R}^{d \times r} $ and $ B \in \mathbb{R}^{r \times k} $, where $ r \ll \min(d, k) $.  
   These matrices approximate the update to $ W $ as:
   $$
   \Delta W = A \cdot B
   $$

   Here, $ r $, the rank of the decomposition, is a key hyperparameter that determines the trade-off between computational cost and model capacity.

3. **Adaptation**:
   During training, instead of updating $ W $, the adapted weight is:
   $$
   W' = W + \Delta W = W + A \cdot B
   $$
   Only the low-rank matrices $ A $ and $ B $ are optimized, while $ W $ remains fixed.

4. **Efficiency**:
   Since $ r $ is much smaller than $ d $ and $ k $, the number of trainable parameters in $ A $ and $ B $ is significantly less than in $ W $. This makes the approach highly efficient both in terms of computation and memory.

---

###  **Fine-Tuning TinyBERT**

For this part, we will fine-tune **TinyBERT**, a distilled version of BERT, using the LoRA method.

- **What is TinyBERT?**

TinyBERT is a lightweight version of the original BERT model created through knowledge distillation. It significantly reduces the model size and inference latency while preserving much of the original BERT’s effectiveness. Here are some key characteristics of TinyBERT:
- It is designed to be more resource-efficient for tasks such as classification, question answering, and more.
- TinyBERT retains a compact structure with fewer layers and parameters, making it ideal for fine-tuning with limited computational resources.


> Similar to the previous section, training this model might take some time. Given the resource limitations, you can train the model for just **2-3 epochs** to demonstrate the process.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType

model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_mapping))

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence Classification
    r=8,                         # Rank of the low-rank matrices A and B
    lora_alpha=32,               # Scaling factor
    lora_dropout=0.1,            # Dropout regularization
    target_modules=["query", "value"]  # Only apply LoRA to attention layers
)

# Apply LoRA configuration to the base model
lora_model = get_peft_model(base_model, lora_config)


In [ ]:
lora_model = get_peft_model(base_model, lora_config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_model.to(device)

trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
print(f"Total Trainable Parameters: {trainable_params}")

optimizer = AdamW(lora_model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 2


Total Trainable Parameters: 10643

In [ ]:
for epoch in range(num_epochs):
    lora_model.train()
    total_loss = 0
    correct = 0
    total_samples = 0

    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = lora_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total_samples
    print(f"Training Loss: {avg_loss:.4f}, Training Accuracy: {accuracy:.4f}")

lora_model.eval()
correct = 0
total_samples = 0
total_loss = 0

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = lora_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

test_loss = total_loss / len(test_loader)
test_accuracy = correct / total_samples
print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/2
100%|██████████| 11811/11811 [24:55<00:00,  7.90it/s]
Training Loss: 1.4220, Training Accuracy: 0.4342

Epoch 2/2
100%|██████████| 11811/11811 [24:54<00:00,  7.90it/s]
Training Loss: 1.2241, Training Accuracy: 0.5122

100%|██████████| 2953/2953 [05:10<00:00,  9.50it/s]

Test Loss: 1.1543, Test Accuracy: 0.5352
